In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 56 kB/s 
     |████████████████████████████████| 200 kB 68.4 MB/s 
     |████████████████████████████████| 199 kB 87.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=ad54161c1cce6aed333e9d14f1ffcb2bcf2c20d71e13e370e3cbc04fef4e3376
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [53]:
from pyspark.sql import SparkSession

In [54]:
spark = SparkSession.builder.appName('aggs').getOrCreate()

In [55]:
df = spark.read.csv("/content/drive/MyDrive/SparkWork/SparkDoc/sales_info.csv", inferSchema=True, header=True)

In [56]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [57]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [58]:
df.describe().show()

+-------+-------+-------+------------------+
|summary|Company| Person|             Sales|
+-------+-------+-------+------------------+
|  count|     12|     12|                12|
|   mean|   null|   null| 360.5833333333333|
| stddev|   null|   null|250.08742410799007|
|    min|   APPL|  Chris|             120.0|
|    max|   MSFT|Vanessa|             870.0|
+-------+-------+-------+------------------+



In [59]:
df.head(3)[2]

Row(Company='GOOG', Person='Frank', Sales=340.0)

### ***GroupBY and Aggregate functions***



In [60]:
df.groupBy('Company').mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [61]:
df.groupBy('Company').min().show()

+-------+----------+
|Company|min(Sales)|
+-------+----------+
|   APPL|     130.0|
|   GOOG|     120.0|
|     FB|     350.0|
|   MSFT|     124.0|
+-------+----------+



In [62]:
df.groupBy('Company').max().show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [63]:
df.groupBy('Company').count().show()

+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



In [65]:
df.groupBy('Company').sum().show()

+-------+----------+
|Company|sum(Sales)|
+-------+----------+
|   APPL|    1480.0|
|   GOOG|     660.0|
|     FB|    1220.0|
|   MSFT|     967.0|
+-------+----------+



In [66]:
df.agg({'sales':'sum'}).show()

+----------+
|sum(sales)|
+----------+
|    4327.0|
+----------+



In [67]:
df.agg({'sales':'min'}).show()

+----------+
|min(sales)|
+----------+
|     120.0|
+----------+



In [68]:
df.agg({'sales':'max'}).show()

+----------+
|max(sales)|
+----------+
|     870.0|
+----------+



In [69]:
df.agg({'sales':'count'}).show()

+------------+
|count(sales)|
+------------+
|          12|
+------------+



In [70]:
df.agg({'sales':'mean'}).show()

+-----------------+
|       avg(sales)|
+-----------------+
|360.5833333333333|
+-----------------+



In [71]:
group_result = df.groupBy('Company')

In [73]:
group_result.agg({'sales':'max'}).show()

+-------+----------+
|Company|max(sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



***Importing some functions from spark***

In [74]:
from pyspark.sql.functions import countDistinct, avg, stddev

**selecting data using select and the above mentioned functions**

In [76]:
df.select(countDistinct('Sales')).show()

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+



alias can be used for aliasing a column header/name

In [78]:
df.select(countDistinct('Company').alias('Number of companies')).show()

+-------------------+
|Number of companies|
+-------------------+
|                  4|
+-------------------+



In [79]:
df.select(avg('sales').alias('Average Sales This Year')).show()

+-----------------------+
|Average Sales This Year|
+-----------------------+
|      360.5833333333333|
+-----------------------+



In [85]:
df.select('sales', 'company').show()

+-----+-------+
|sales|company|
+-----+-------+
|200.0|   GOOG|
|120.0|   GOOG|
|340.0|   GOOG|
|600.0|   MSFT|
|124.0|   MSFT|
|243.0|   MSFT|
|870.0|     FB|
|350.0|     FB|
|250.0|   APPL|
|130.0|   APPL|
|750.0|   APPL|
|350.0|   APPL|
+-----+-------+



### ***Sorting using OrderBY***


*   Ascending
*   Descending



In [88]:
df.orderBy('sales').show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [90]:
df.orderBy(df['sales'].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+

